In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
!pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="TheBloke/Mistral-7B-OpenOrca-GGUF",
	filename="mistral-7b-openorca.Q2_K.gguf",
)


In [ ]:
!pip install chromadb

In [ ]:
import chromadb
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/chromadb_text_lab5")

In [ ]:
collection = chroma_client.get_collection(name="text")
collection.count()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
ch_text = pd.read_csv('/content/drive/MyDrive/chunked_text.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
ch_t = ch_text[['id', 'chunked_text']]
ch_t = ch_text.set_index('id')

In [ ]:
def get_answers(collection, ch_text, q):
    answ = []

    q_e = model.encode(q)
    que = collection.query(
        query_embeddings=q_e,
        include=["documents"]
    )
    res = que['ids'][0][:3]

    for id in res:
        answ.append(ch_text.loc[id]['chunked_text'])

    return answ


In [ ]:
def get_responce(collection, ch_t, q):
    context = get_answers(collection, ch_t, q)
    context = ' '.join(context)
    template = f"Answer the user query using the context. User: {q} considering Context {context}"

    resp = ''
    att = 0

    while resp == '' and att < 10:
        att += 1
        r = llm.create_chat_completion(
            messages = [
                {
                    "role": "user",
                    "content": template
                }
            ]
        )

        resp = r['choices'][0]['message']['content'].replace('\n', '')


    return resp


In [ ]:
q = 'Which Harry Potter movie would you recommend?'
r = get_responce(collection, ch_t, q)

In [ ]:
q = 'About what Harry Potter and the Half-Blood Prince?'

r = get_responce(collection, ch_t, q)
r

In [ ]:
!pip install --upgrade gradio

In [ ]:
import gradio as gr

def chat(q):
    r = get_responce(collection, ch_t, q)
    print(r)
    return r

demo = gr.Interface(
    fn=chat,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()


In [ ]:
questions = [
    'Which Harry Potter movie is the most popular?',
    'How much should pay a singer to perform in US?',
    'How many songs iTunes have sold?',
    'What was the first film Johny Depp is played?',
    'Who will be performing in 25th award ceremony in London?',
    'In which movie played Keanu Reeves?',
    'Best childrens film due to Bafta?',
    'When Van Gogh was stabbed to death?',
    'In which films Lee was Co-creator?',
    'How many nominations for the awards has Alicia Keys?'
]

In [ ]:
responces = []


for q_i in tqdm(questions):
    r_i = get_responce(collection, ch_t, q_i)
    responces.append(r_i)

In [ ]:
q = 'How many nominations for the awards has Alicia Keys?'

r = get_responce(collection, ch_t, q)

In [ ]:
references = [
    'Harry Potter and the Prisoner of Azkaban, runner-up in the poll, was the choice of the Bafta experts who named it best feature film.',
    'A singer hoping to perform in the US can expect to pay $1,300 simply for obtaining a visa.',
    'More than 200 million songs have been sold by the iTunes music store since it was launched.',
    'In 1984 he started his film career playing a doomed teenager in horror film Nightmare On Elm Street.',
    'Ms Stone is due to perform at the 25th award ceremony at Londons Earls Court.',
    'Keanu Reeves given Hollywood star  Actor Keanu Reeves, best known for his role in the Matrix movies, has been awarded a star on the prestigious Hollywood Walk of Fame.',
    'Children vote Shrek 2 best film  Young UK film fans voted animated Hollywood hit Shrek 2 best film at the childrens Bafta awards on Sunday.',
    'Van Gogh was shot and stabbed in November 2004, following death threats he received about his film Submission.',
    'As well as Spider-Man, Lee co-created the Incredible Hulk, X-Men, Daredevil and Fantastic Four characters.',
    'while Alicia Keys has three.'
]

In [ ]:
!pip install evaluate

In [ ]:
!pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")
results = bertscore.compute(predictions=responces, references=references, lang="en")

In [ ]:
results